<a href="https://colab.research.google.com/github/lizbethlulu/Research-/blob/main/new_SBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('distiluse-base-multilingual-cased')
import numpy as np

from sentence_transformers import SentenceTransformer, models

# Define the model name for Indonesian sentiment analysis
model_name = 'ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa'

# Configure the SentenceTransformer model
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])



In [ ]:
# df = pd.read_csv("https://drive.google.com/uc?export=download&id=1y8MlToeRw6avPU7qN47p3zIhhQuWKqMs")
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Xv14IoQjRKuYhji5Zadj4TD1SN8vkIdJ")


df

In [ ]:
dicts = df.to_dict(orient='list')
dicts

In [ ]:
# import os


In [ ]:
# def import_data(csv_file_path):
#     datas = {}
#     if os.path.exists(csv_file_path):
#         # Read data from CSV
#         df = pd.read_csv(csv_file_path)
#         datas = df.to_dict(orient='list')
#         print("Columns in the CSV file:", list(datas.keys()))
#     else:
#         print("File does not exist. Creating a new dictionary.")
#     return datas

In [ ]:
# csv_file_path = "/content/Fix_data.csv"
# print(csv_file_path)
# dicts = import_data("Fix_data.csv")

In [ ]:
len(dicts.keys())
dicts.keys()

In [ ]:
from tqdm import tqdm
# datas = {}
# for key in tqdm(dicts.keys(),"Sentence Embedding"):
#     tmp = dicts[key]['label'].copy()
#     tmp.extend(dicts[key]['comments'])
#     # tmp = df[df["label"] == key]["comments"].tolist()
#     # tmp.extend(df[df["label"] == key]["comments"].tolist())
#     if len(tmp) == 0:
#         print(f"label {key} has no posts or tweets")
#         continue
#     arr = model.encode(tmp[0]).reshape(1, -1)
#     for t in tmp[1:]:
#         brr = model.encode(str(t))
#         arr = np.concatenate((arr, brr.reshape(1, -1)), axis=0)
#         print(f"Finished encoding tweet from label {key}")
#     datas[key] = np.mean(arr, axis=0)

# datas = {}
# for i in tqdm(range(len(dicts['label'])), "Sentence Embedding"):
#     label = dicts['label'][i]
#     comment = dicts['comments'][i]
#     if not comment:
#         print(f"Label {label} has no posts or tweets")
#         continue
#     arr = model.encode(comment).reshape(1, -1)
#     if label in datas:
#         datas[label] = np.concatenate((datas[label], arr), axis=0)
#     else:
#         datas[label] = arr
#     print(f"Finished encoding tweet for label {label}")

# for label in datas:
#     datas[label] = np.mean(datas[label], axis=0)

# print(datas)

datas = {}
for i in tqdm(range(len(dicts['label'])), "Sentence Embedding"):
    label = dicts['label'][i]
    comment = dicts['comments'][i]
    if not comment:
        print(f"Label {label} has no posts or tweets")
        continue
    arr = model.encode(comment).reshape(1, -1)
    if label in datas:
        datas[label].append(arr)
    else:
        datas[label] = [arr]
    print(f"Finished encoding tweet for label {label}")

# print(datas)

In [ ]:
datas

In [ ]:
# encoded_datas = datas[list(datas.keys())[0]].reshape(1, -1)
# for label in list(datas.keys())[1:]:
#     encoded_datas = np.concatenate((encoded_datas, datas[label].reshape(1, -1)), axis=0)

for label in datas:
    datas[label] = np.array(datas[label])

# Menggabungkan semua numpy array menjadi satu array besar
encoded_datas = np.concatenate(list(datas.values()), axis=0)


In [ ]:
print(len(list(datas.keys())),encoded_datas.shape)
encoded_datas

In [ ]:
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score as ss

In [ ]:
pca = PCA(n_components=2)
result = pca.fit_transform(encoded_datas)

In [ ]:
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
sns.scatterplot(x=[X[0] for X in result],
                y=[X[1] for X in result],
                palette="deep",
                legend=None
                )
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
clustering = DBSCAN(eps=0.2, min_samples=2).fit(encoded_datas)

In [ ]:
labels = clustering.labels_
N_clus=len(set(labels))-(1 if -1 in labels else 0)
print('Estimated no. of clusters: %d' % N_clus)

# Identify Noise
n_noise = list(clustering.labels_).count(-1)
print('Estimated no. of noise points: %d' % n_noise)
ss(encoded_datas, labels)

In [ ]:
epsilon = np.linspace(0.1,1,num=5)
epsilon
min_samples = np.arange(2,10,step=3)
min_samples

In [ ]:
import itertools
combinations = list(itertools.product(epsilon,min_samples))
combinations

In [ ]:
all_parameters = []

def get_scores_and_labels(combinations, encoded_datas):
    scores = []
    all_labels_list = []
    n = len(combinations)  # Added to get the number of combinations for print statements
    for i, (eps, num_samples) in enumerate(combinations):
        clustering = DBSCAN(eps=eps, min_samples=num_samples).fit(encoded_datas)
        labels = clustering.labels_
        labels_set = set(labels)
        num_clusters = len(labels_set)
        if -1 in labels_set:
            num_clusters -= 1
        if (num_clusters < 2) or (num_clusters > 50):
            scores.append(-10)
            all_labels_list.append("bad")
            c = (eps, num_samples)
            print(f"combination {c} on iteration {i+1} of {n} has {num_clusters} clusters. next!")
            continue
        ss_val = ss(encoded_datas, labels)
        outliers = np.sum(labels == -1)
        scores.append(ss_val)
        all_labels_list.append(labels)
        parameters = {
            'epsilon': eps,
            'min_samples': num_samples,
            'score': ss_val,
            'num_clusters': num_clusters,
            'outliers': outliers
        }
        all_parameters.append(parameters)
        print(f"index {i}, score: {scores[-1]}, labels: {all_labels_list[-1]}, numClusters: {num_clusters}")
        parameters
    best_index = np.argmax(scores)
    best_parameters = combinations[best_index]
    best_labels = all_labels_list[best_index]
    best_scores = scores[best_index]
    return {
        'best_epsilon': best_parameters[0],
        'best_min_samples': best_parameters[1],
        'best_labels': best_labels,
        'best_score': best_scores
    }

best_dic = get_scores_and_labels(combinations,encoded_datas)

In [ ]:
pr = pd.DataFrame(all_parameters)
pr

In [ ]:
# encoded_datas[0]
best_dic
# df['cluster'] = best

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(encoded_datas[48].reshape(1,-1),encoded_datas[50].reshape(1,-1))

In [ ]:
df['cluster'] = best_dic['best_labels']
df
df['cluster'].value_counts()

In [ ]:
labels = df['cluster']
# labels
clusters = {}
for label in set(labels):
    clusters[label] = df[df['cluster'] == label]['comments'].tolist()
    # print(f"labels : {label}, clusters :  {len(clusters[label])}")

clusters[-1]

In [ ]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary

def apply_lda_to_clusters(clusters, num_topics):
    lda_models = {}
    for label, documents in clusters.items():
        print(f"Processing cluster {label} with {len(documents)} documents")
        if len(documents) == 0:
            print(f"Skipping cluster {label} because it is empty")
            continue
        tokenized_docs = [doc.split() for doc in documents]
        dictionary = Dictionary(tokenized_docs)
        if len(dictionary) == 0:
            print(f"Skipping cluster {label} because dictionary is empty")
            continue
        corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]
        if len(corpus) == 0:
            print(f"Skipping cluster {label} because corpus is empty")
            continue  # Skip clusters with empty corpus
        # Melatih model LDA
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
        lda_models[label] = {
            'lda_model': lda_model,
            'corpus': corpus,
            'dictionary': dictionary
        }
        print(f"Finished processing cluster {label}")
    return lda_models

# Terapkan LDA pada setiap kelompok dokumen
num_topics = 1
lda_models = apply_lda_to_clusters(clusters, num_topics=num_topics)


In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def display_topics_and_wordcloud(lda_model, num_topics, num_words=10):
    for topic_num in range(num_topics):
        topic_words = lda_model.show_topic(topic_num, topn=num_words)
        topic_words_str = ", ".join([f"{word} ({prob:.9f})" for word, prob in topic_words])
        print(f"Topik {topic_num}: {topic_words_str}")
        word_freq = {word: prob for word, prob in topic_words}
        wordcloud = WordCloud(width=800, height=400, background_color='black').generate_from_frequencies(word_freq)
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Topic {topic_num + 1}')
        plt.show()

# Penggunaan:
for label, model_info in lda_models.items():
    print(f"Topik untuk kluster {label}:")
    display_topics_and_wordcloud(model_info['lda_model'], num_topics=num_topics, num_words=900)
    print("\n")

In [ ]:
cluster_distribution = df.groupby(['sosmed', 'cluster']).size().unstack(fill_value=0)
print(cluster_distribution)